In [4]:
# !pip3 install typeguard

In [5]:
from herpetologist import check_type
from typeguard import typechecked
from typing import List, Tuple, Dict

In [16]:
import herpetologist
import typeguard
herpetologist.__version__

'0.0.9'

### Simple checking

In [7]:
@typechecked
def greeting(name: str):
    print(name)
    
greeting('foo') # no error
greeting(1) # error

foo


TypeError: type of argument "name" must be str; got int instead

In [8]:
@check_type
def greeting(name: str):
    print(name)
    
greeting('foo') # no error
greeting(1) # error

foo


TypeError: type of argument "name" from greeting(name) must be 'str'

In [20]:
@typechecked
def greeting(name: List[str]):
    print(name)
    
greeting(['foo']) # no error
greeting([1]) # error

['foo']


TypeError: type of argument "name"[0] must be str; got int instead

### Complex checking

In [26]:
@typechecked
def greeting(name: Tuple[Tuple[List[Dict[int, str]]], Tuple[List[int]]]):
    print(name)
    
greeting((([{1: 'str'}],), ([1], [1]))) # error
greeting((([{1: 'str'}],), ([1], [1.1]))) # error

TypeError: argument "name"[1] has wrong number of elements (expected 1, got 2 instead)

In [27]:
@check_type
def greeting(name: Tuple[Tuple[List[Dict[int, str]]], Tuple[List[int]]]):
    pass

greeting((([{1: 'str'}],), ([1], [1]))) # no error
greeting((([{1: 'str'}],), ([1], [1.1]))) # error

TypeError: type of argument "name" from greeting(name) must be Tuple[Tuple[List[Dict[int, str]]], Tuple[List[int]]]

In [28]:
@typechecked
def greeting(name: List[str]):
    pass

greeting(['h']) # no error
greeting([1]) # error

TypeError: type of argument "name"[0] must be str; got int instead

In [29]:
@check_type
def greeting(name: List[Tuple[str, int, float]]):
    pass

greeting([('h', 1, 1.1)]) # no error
greeting([('h', 1, 1)]) # error

TypeError: type of argument "name" from greeting(name) must be List[Tuple[str, int, float]]

In [31]:
%%time

@typechecked
def greeting(name: List[Tuple[str, int, float]]):
    pass

greeting([('h', 1, 1.1)]) # no error
greeting([('h', 1, 1)]) # no error, supposedly error

CPU times: user 351 µs, sys: 1e+03 ns, total: 352 µs
Wall time: 357 µs


In [35]:
@check_type
def greeting(name: List[Tuple[str, int, float]]):
    pass

greeting([('h', 1, 1.1)]) # no error
greeting([('h', 1, 1)]) # error

TypeError: type of argument "name" from greeting(name) must be List[Tuple[str, int, float]]

In [36]:
@check_type
def greeting(name: Tuple[Tuple[List[Dict[int, str]]], Tuple[List[int]]]):
    pass

greeting((([{1: 'str'}],), ([1], [1]))) # no error
greeting((([{1: 'str'}],), ([1], [1.1]))) # error

TypeError: type of argument "name" from greeting(name) must be Tuple[Tuple[List[Dict[int, str]]], Tuple[List[int]]]

In [37]:
@check_type
def greeting(name: List[Dict[str, Dict[str, List[str]]]]):
    pass
    
greeting([{'a': {'b': ['a', 'a']}}]) # no error
greeting([{'a': {'b': ['a', 1]}}]) # error

TypeError: type of argument "name" from greeting(name) must be List[Dict[str, Dict[str, List[str]]]]

In [38]:
from test_class import A

In [39]:
a = A()

In [40]:
@check_type
def greeting(name: Tuple[Tuple[List[Dict[int, A]]], Tuple[List[int], float, Tuple[Dict[str, str]]]]):
    pass

greeting((([{1: a}],), ([1], 1.1, ({'a': 'a'},)))) # no error
greeting((([{1: 'str'}],), ([1], 1.1, ({'a': 'a'}, {'a': 1})))) # error

TypeError: type of argument "name" from greeting(name) must be Tuple[Tuple[List[Dict[int, test_class.A]]], Tuple[List[int], float, Tuple[Dict[str, str]]]]

## default value

herpetologist will never checked default value, snake is a snake (concept from duck typing)!

In [41]:
@check_type
def greeting(name: List[Dict[str, Dict[str, List[str]]]] = None):
    if name is None:
        print('name is None!')
    
greeting([{'a': {'b': ['a', 'a']}}]) # no error
greeting() # no error

name is None!


## disable herpetologist using bash environment

Some of our environments we want to enable it, some of it also we want to disable it, because we do not want herpetologist to keep check the variables. So to disable it, simply set bash environment.

In [3]:
@check_type
def plus(x: List[int]):
    return [i + 1 for i in x]

def plus_nocheck(x: List[int]):
    return [i + 1 for i in x]

We have a simple function, and if you check herpetologist source code, it checks variables recursively. So this will become expensive if we have a huge variables.

In [5]:
x = [i for i in range(100000)]

In [9]:
%%time

r = plus(x)

CPU times: user 18.4 ms, sys: 1.75 ms, total: 20.2 ms
Wall time: 20.2 ms


In [10]:
%%time

r = plus_nocheck(x)

CPU times: user 7.43 ms, sys: 1.58 ms, total: 9.01 ms
Wall time: 9.53 ms


**different almost 11 ms, that is a lot.**

So to disable herpetologist without need to change the source code, we can use bash environment. **But again, this will disable entire herpetologist.**

In [11]:
import os
os.environ['ENABLE_HERPETOLOGIST'] = 'false'

In [13]:
import importlib
import herpetologist
importlib.reload(herpetologist)

<module 'herpetologist' from '/Users/huseinzolkepli/Documents/herpetologist/herpetologist/__init__.py'>

In [14]:
herpetologist.enable_check

False

In [15]:
%%time

r = plus(x)

CPU times: user 7.26 ms, sys: 1.52 ms, total: 8.78 ms
Wall time: 9.26 ms
